<a href="https://colab.research.google.com/github/DmitryKutsev/cross-document_coreference/blob/main/classifiers/rule_ml_pair_pron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Импорт**

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.display import SVG, display, HTML

In [3]:
from google.colab import drive
drive.mount('./content', force_remount=False)

Mounted at ./content


In [4]:
import spacy
import numpy as np

import gensim
from gensim.models import Word2Vec 
from gensim.models import FastText

from nltk.tokenize import RegexpTokenizer
from nltk import  sent_tokenize
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
from graphviz import Source
from sklearn import tree, svm, linear_model
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


In [6]:
!ls
import os
for i in os.listdir('content/'):
  print(i)

content  sample_data
.file-revisions-by-id
.shortcut-targets-by-id
MyDrive
.Trash


In [7]:
import csv
import re
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

In [8]:
pip install pymorphy2[fast]

     |████████████████████████████████| 61kB 6.0MB/s 
     |████████████████████████████████| 8.2MB 1.4MB/s 
     |████████████████████████████████| 378kB 34.1MB/s 
  Created wheel for DAWG: filename=DAWG-0.8.0-cp37-cp37m-linux_x86_64.whl size=860466 sha256=1d82de97dd4d2ce9ab8f610d8b485701ea886dcf5269fcb237058ccb3ddf28c6
  Stored in directory: /root/.cache/pip/wheels/3d/1f/f0/a5b1f9d02e193c997d252c33d215f24dfd7a448bc0166b2a12
Successfully built DAWG


## **Обработка данных**

In [9]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/hv-info_gepatit2.txt

--2021-05-30 22:00:22--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/neuroleptic_antidepr2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11655409 (11M) [text/plain]
Saving to: ‘neuroleptic_antidepr2.txt’

neuroleptic_antidep 100%[===================>]  11.12M  49.6MB/s    in 0.2s    

2021-05-30 22:00:23 (49.6 MB/s) - ‘neuroleptic_antidepr2.txt’ saved [11655409/11655409]

--2021-05-30 22:00:23--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/scraping/rakpobedim_ru2.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443.

In [10]:
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
!wget https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv

--2021-05-30 22:00:25--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/coref_df_anns60k.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2021-05-30 22:00:25 ERROR 404: Not Found.

--2021-05-30 22:00:26--  https://raw.githubusercontent.com/DmitryKutsev/cross-document_coreference/main/data/annotation_table.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2100469 (2.0M) [text/plain]
Saving to: ‘annotation_table.csv’

annotation_table.cs 100%[===================>]   2.00M  --.-KB/s    in 0.04

In [11]:
handler = open('neuroleptic_antidepr2.txt', 'r', encoding='utf-8')
my_txt = handler.read()
handler.close()

handler = open('hv-info_gepatit2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

handler = open('rakpobedim_ru2.txt', 'r', encoding='utf-8')
my_txt += handler.read()
handler.close()

In [12]:
posts = my_txt.split('----')

In [13]:
import pymorphy2
morph = pymorphy2.MorphAnalyzer()

In [14]:
morph.parse('он')[0].normal_form

'он'

## **Обучение векторизатора**

In [15]:
tokenizer = RegexpTokenizer(r'\w+')
sentences_tokenized = [w.lower() for w in posts]


sentences_tokenized = [tokenizer.tokenize(morph.parse(i)[0].normal_form) for i in sentences_tokenized]
ft = FastText(sentences_tokenized, window=5, min_count=5, workers=4,sg=1)

Обработка битых символов, чтобы pandas мог открыть таблицу.

In [16]:
count = 0
with open('annotation_table.csv') as csvfile:
  with open('annotations.csv', 'w') as write_file:
    writer = csv.writer(write_file, delimiter='\t')
    for new_row in csvfile:
      count +=1
      # print(i.split('\t'))
      if len(new_row.split('\t')) > 19:
        new_row = new_row.split('\t')[:20]
        new_row[-1] = new_row[-1][:-1] # cut \n
      writer.writerow(new_row)
count

27019

In [17]:
# # with open('coref_df_anns60k.csv') as csvfile:
# with open('new_anns60k.csv') as csvfile:
#   reader = csv.reader(csvfile, delimiter='\t')
#   with open('new_df_anns60k.csv', 'w') as write_file:
#     writer = csv.writer(write_file, delimiter='\t')
#     c = 0
#     for row in reader:
#       c +=1
#       new_row = [i if i and not re.match(r'\t+?', i) else 0 for i in row]
#       if len(new_row) > 9:
#         new_row = new_row[:9]
#       else:
#         new_row = [0 for i in range(9)]
#       writer.writerow(new_row)
# c


In [18]:
# !rm new_anns60k.csv.1 new_anns60k.csv.2 new_df_anns60k.csv new_anns60k.csv
!ls
# !rm annotation_table.csv.2 annotation_table.csv.1 annotation_table.csv annotations.csv

annotations.csv       hv-info_gepatit2.txt	 sample_data
annotation_table.csv  neuroleptic_antidepr2.txt
content		      rakpobedim_ru2.txt


In [19]:
# anns_df = pd.read_csv('coref_df_anns60k.csv', delimiter='\t', error_bad_lines=False, header=None)
anns_df = pd.read_csv('annotations.csv', delimiter='\t')
anns_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27013,и,0,22,68.0,0.0,2385.0,NaN,NaN,NaN,NaN,CONJ,cc,пошлины,inan,nomn,femn,NOUN,0,0.0,NaN
27014,таможенные,0,22,68.0,0.0,2386.0,NaN,nomn,NaN,NaN,ADJF,amod,пошлины,inan,nomn,femn,NOUN,0,0.0,plur
27015,пошлины,0,22,68.0,0.0,2387.0,inan,nomn,femn,NaN,NOUN,nsubj,есть,NaN,NaN,NaN,INFN,0,0.0,plur
27016,как,0,22,68.0,0.0,2388.0,NaN,NaN,NaN,NaN,CONJ,advmod,есть,NaN,NaN,NaN,INFN,0,0.0,NaN


In [20]:
tok_list = []
for token in anns_df.TOKEN:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['TOKEN_VECT'] = tok_list


tok_list = []
for token in anns_df.HEAD:
  try:
    tok_list.append(ft.wv.__getitem__(token.lower()))
  except (KeyError, AttributeError):
    tok_list.append(0)
    # print(token, 'here')
anns_df['HEAD_VECT'] = tok_list

In [21]:
anns_df = anns_df.loc[(anns_df['TOPIC_NUM'].isin(list(range(30)))) | (anns_df['SENT_NUM'].isin(list(range(10000))))]
anns_df = anns_df.reset_index().drop(['index'], axis=1)

In [22]:
topics = list(set(anns_df.TOPIC_NUM.values))
topics
sent_vect_list = []
# номера предложений топика
for topic_num in topics:

  sents_in_topic = list(set(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num].SENT_NUM))
  topic_len = len(anns_df.loc[anns_df['TOPIC_NUM'] == topic_num])
  for sent_num in sents_in_topic:
    tok_vects = anns_df.loc[(anns_df['TOPIC_NUM'] == topic_num) & (anns_df['SENT_NUM'] == sent_num)].TOKEN_VECT
    try:
      sent_vect = sum(list(tok_vects))/len(tok_vects)
    except ZeroDivisionError:
      sent_vect = 0
      print(anns_df.loc[(anns_df['SENT_NUM'] == sent_num)])
    for i in range(len(tok_vects)):
      sent_vect_list.append(sent_vect)
len(sent_vect_list)
# если не класть нулевых предложений - будет длина датафрейма, как он раньше импортировлся, но поздно.

26497

In [23]:
anns_df['SENT_VECT'] = sent_vect_list

Проверим данные

In [24]:
anns_df.SENT_NUM[18]

1.0

# **VECTORS HANDLING**

Изменяем значения векторов предложений, главного слова группы, зависимых слов группы, и токена на косинусную дистанцию с семантической осью.

Сначала, изменим строковое представление на массив.

Добавим размеченные данные, и посмотрим, что получилось.

Ограничим датафрейм теми данными, которые были размечены, чтобы была возможность провести оценку системы.

In [25]:
# df = anns_df.iloc[:15000,:]
# df = anns_df.iloc[:17000,:]
df = anns_df.iloc[:18000,:]

Всего наименований препаратов размечено (это не обязательно антецеденты, любое название препарата).

In [26]:
len(df.loc[df['NER'] == 1])

628

Из них местоимений (прономинальные анафоры).

In [27]:
len(df.loc[df['NER'] == 3])

117

И номинативные анафоры существительные, такие как "препарат" и "лекарство". В основном это они и есть.

In [28]:
len(df.loc[df['NER'] == 2])

246

In [29]:
len(df.loc[(df['NER'].isin([2,3]))])

363

In [30]:
df.loc[df['COREFERENCE_CLUSTER']==0]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,"[-0.31890675, 0.5212655, -0.32741612, -0.03404...","[-0.31890675, 0.5212655, -0.32741612, -0.03404...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,"[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[-0.31890675, 0.5212655, -0.32741612, -0.03404...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
5,не,0,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,"[0.5272131, -0.051245436, -0.7849624, 0.202875...","[1.2486959, 0.18037225, 0.498966, -0.329809, -...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
6,знаю,0,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[1.2486959, 0.18037225, 0.498966, -0.329809, -...","[1.2486959, 0.18037225, 0.498966, -0.329809, -...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
7,стоило,0,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,"[0.35586137, 0.48749256, 0.8564054, 0.05411040...","[1.2486959, 0.18037225, 0.498966, -0.329809, -...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,"[0.6379133, 0.43270725, 0.15673968, 0.04011524...","[0.54932636, 0.81035197, 0.54798865, 0.6713344...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,"[0.019343415, 0.53147113, 0.3195495, 0.4243641...","[0.6379133, 0.43270725, 0.15673968, 0.04011524...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,"[0.11613229, 0.04169186, -0.5249635, 0.1126221...","[-0.21995926, 0.13575025, 0.8849688, 0.6176648...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,"[-0.34641692, 0.4395416, 0.30565873, -0.183525...","[0.049758185, 0.49402085, 1.0832764, -0.190180...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."


In [31]:
# df['NER'] = [int(i) for i in df['NER']]
# df['COREFERENCE_CLUSTER'] = [int(i) for i in df['COREFERENCE_CLUSTER']]

Посмотрим на данные.

In [32]:
df.loc[df['NER'].isin([1,2,3])]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,"[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[-0.31890675, 0.5212655, -0.32741612, -0.03404...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.45032018, 0.3640179, 0.4912018, 0.053628337...","[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,"[0.5007583, 0.48945272, 0.47897717, 0.12774718...","[0.45032018, 0.3640179, 0.4912018, 0.053628337...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,"[0.56919163, 0.6927173, 0.53424066, -0.1588261...","[0.5007583, 0.48945272, 0.47897717, 0.12774718...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
21,лекарством,0,1,1.0,1.0,21.0,inan,ablt,neut,NaN,NOUN,conj,нет,NaN,NaN,NaN,PRED,2,0.0,sing,"[-0.23204534, 0.8725322, 0.78436136, 0.4096064...","[0.57206756, 0.0030588252, -0.15605439, 0.3244...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,0,21,4.0,2.0,137.0,NaN,NaN,masc,NaN,VERB,ROOT,вел,NaN,NaN,masc,VERB,1,136.0,sing,"[1.1968448, -1.0497357, 0.38124982, 0.31607455...","[1.1968448, -1.0497357, 0.38124982, 0.31607455...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."
17929,соф,0,21,4.0,3.0,148.0,inan,gent,femn,NaN,NOUN,nmod,форума,inan,gent,masc,NOUN,1,148.0,plur,"[0.80214727, -0.67269385, 0.44578657, -0.29348...","[1.233638, -0.0804277, -0.13442877, -0.5695290...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."
17930,даке,0,21,4.0,3.0,149.0,inan,datv,femn,NaN,NOUN,nmod,соф,inan,gent,femn,NOUN,1,149.0,sing,"[0.40923402, -0.61117554, 0.1905692, 0.3549277...","[0.80214727, -0.67269385, 0.44578657, -0.29348...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."
17937,соф,0,21,4.0,3.0,156.0,inan,gent,femn,NaN,NOUN,nsubj,вел,NaN,NaN,masc,VERB,1,156.0,plur,"[0.80214727, -0.67269385, 0.44578657, -0.29348...","[1.1968448, -1.0497357, 0.38124982, 0.31607455...","[0.27393702, 0.35870212, 0.23169681, -0.120225..."


# **SEMANTIC AXIS**

Реализация метода семантических осей.
Сначала, выбираем первые предложения каждого топика. Посмотрим на них.

In [33]:
axis_df = df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0')
axis_df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.45032018, 0.3640179, 0.4912018, 0.053628337...","[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[0.32353708, 0.52701545, 0.28991348, -0.262836..."
576,зелдокс,0,2,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,"[0.47048634, 0.67670745, 0.33279476, 0.398212,...","[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[0.4020621, 0.34669644, 0.19212848, -0.0557638..."
1870,оланзапин,0,3,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.42822498, 0.43878368, 0.39959782, 0.0953036...","[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[0.28177658, 0.4777933, 0.20035708, -0.2101104..."
5555,галоперидол,0,4,0.0,0.0,2.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,1,2.0,sing,"[0.5054167, 0.5160975, 0.21932103, 0.1581231, ...","[-0.31890675, 0.5212655, -0.32741612, -0.03404...","[0.24623537, 0.26800618, 0.2504264, -0.2448514..."
6200,тиаприд,0,5,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,3.0,sing,"[0.42402324, 0.41187572, 0.59262645, 0.1015688...","[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[0.28907463, 0.20248425, 0.26165906, -0.234126..."
8450,сероквель,0,6,0.0,0.0,2.0,inan,gent,femn,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,plur,"[0.5557974, 0.5513919, 0.6032676, 0.12429671, ...","[-0.28692204, 0.4596792, 0.6769302, 0.95611686...","[0.33045644, 0.27588293, 0.20549099, -0.190540..."
8541,день,0,7,0.0,0.0,2.0,inan,accs,masc,NaN,NOUN,ROOT,день,inan,accs,masc,NOUN,0,0.0,sing,"[-0.22351742, 1.0214677, -0.39858544, 0.383429...","[-0.22351742, 1.0214677, -0.39858544, 0.383429...","[0.33045644, 0.27588293, 0.20549099, -0.190540..."
11117,500,0,8,0.0,0.0,2.0,NaN,NaN,NaN,NaN,NaN,nummod,мг,inan,nomn,masc,NOUN,0,0.0,NaN,"[0.17585571, -0.23459816, 0.4998092, -0.168273...","[0.32468337, 0.05964525, 0.31036082, -0.528615...","[0.30224326, 0.26744294, 0.20574696, -0.132465..."
11787,капецитабин,0,10,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,ROOT,капецитабин,anim,nomn,masc,NOUN,1,2.0,sing,"[0.32422826, 0.64003587, 0.3145818, -0.0434836...","[0.32422826, 0.64003587, 0.3145818, -0.0434836...","[0.26897487, 0.2889512, 0.23881891, -0.1320692..."
11806,иресса,0,11,0.0,0.0,2.0,NaN,NaN,NaN,NaN,GRND,nmod,отдам,NaN,NaN,NaN,VERB,1,2.0,NaN,"[0.038471747, 0.7510374, 0.29925403, -0.540814...","[0.91993815, 0.39189303, -0.48954776, -0.09187...","[0.26897487, 0.2889512, 0.23881891, -0.1320692..."


In [34]:
# sum(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)/len(df.query('POST_NUM == 0 and TOKEN_NUM == 2 and SENT_NUM == 0').TOKEN_VECT)

Суммируем верктора токенов, главных слов, предложения целиком, и зависимых слов. Нормализуем.

In [35]:

token_axe = sum(axis_df.TOKEN_VECT)/len(axis_df.TOKEN_VECT)
head_axe = sum(axis_df.HEAD_VECT)/len(axis_df.HEAD_VECT)
# head_childs_axe = sum(axis_df.HEAD_CHILDS_VECT)/len(axis_df.HEAD_CHILDS_VECT)
sent_axe = sum(axis_df.SENT_VECT)/len(axis_df.SENT_VECT)

In [36]:
df.TOKEN_VECT[710]

0

Функция, которая заменяет столбец с векторами на столбец со значениями косинусной дистанции с соответствующей семантической осью.


In [37]:
def make_similarity(df, col, axe):
  new_col = []
  for ind, vect in enumerate(df[col]):
    try:
      new_col.append(cosine_similarity([axe], [vect])[-1][-1])
    except ValueError:
      new_col.append(0)
  df[col] = new_col



In [38]:
make_similarity(df, 'TOKEN_VECT', token_axe)
make_similarity(df, 'HEAD_VECT', head_axe)
# make_similarity(df, 'HEAD_CHILDS_VECT', head_childs_axe)
make_similarity(df, 'SENT_VECT', sent_axe)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [39]:
df.TOKEN[1]

'препарат'

In [40]:
# df['SUM_VECT']= [(df.TOKEN_VECT[i] + df.SENT_VECT[i])/3 for i in range(len(df.SENT_VECT))]

Посмотрим на обработанные данные.

In [41]:
df

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,обсуждаем,0,1,0.0,0.0,0.0,NaN,NaN,NaN,1per,VERB,ROOT,обсуждаем,NaN,NaN,NaN,VERB,0,0.0,plur,0.497393,0.528361,0.947129
1,препарат,0,1,0.0,0.0,1.0,inan,nomn,masc,NaN,NOUN,obj,обсуждаем,NaN,NaN,NaN,VERB,2,0.0,sing,0.602498,0.528361,0.947129
2,рисперидон,0,1,0.0,0.0,2.0,anim,nomn,masc,NaN,NOUN,appos,препарат,inan,nomn,masc,NOUN,1,2.0,sing,0.735588,0.858878,0.947129
3,рисполепт,0,1,0.0,0.0,3.0,inan,gent,femn,NaN,NOUN,parataxis,рисперидон,anim,nomn,masc,NOUN,1,3.0,plur,0.763361,0.562444,0.947129
4,сперидан,0,1,0.0,0.0,4.0,NaN,NaN,masc,NaN,PRTS,conj,рисполепт,inan,gent,femn,NOUN,1,4.0,sing,0.730097,0.628638,0.947129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,схемы,0,21,6.0,0.0,214.0,inan,gent,femn,NaN,NOUN,obj,использовать,NaN,NaN,NaN,INFN,0,0.0,sing,0.434102,0.562817,0.985054
17996,рекомендованные,0,21,6.0,0.0,215.0,NaN,nomn,NaN,NaN,PRTF,acl,схемы,inan,gent,femn,NOUN,0,0.0,plur,0.626010,0.412885,0.985054
17997,в,0,21,6.0,0.0,216.0,NaN,NaN,NaN,NaN,PREP,case,версиях,inan,loct,femn,NOUN,0,0.0,NaN,0.477165,0.490797,0.985054
17998,более,0,21,6.0,0.0,217.0,NaN,NaN,NaN,NaN,ADVB,advmod,ранних,NaN,gent,NaN,ADJF,0,0.0,NaN,0.520455,0.511817,0.985054


# **ПРАВИЛОВЫЙ АЛГОРИТМ**

In [42]:
# df.SENT_NUM = [int(i) for i in df['SENT_NUM']]
# df.POST_NUM = [int(i) for i in df['POST_NUM']]
# # df.TOKEN_NUM = [float(i) for i in df['TOKEN_NUM']]
df.TOPIC_NUM = [int(i) for i in df['TOPIC_NUM']]
df.IS_ANSWER = [int(i) for i in df['TOPIC_NUM']]
# df.NER = [int(i) for i in df['NER']]

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:5170: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [43]:
topics = list(set(df.TOPIC_NUM.values))
topics

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [44]:
df.loc[df['TOKEN_NUM'] == 344 ]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
344,рисполепта,1,1,2.0,4.0,344.0,inan,nomn,femn,NaN,NOUN,nmod,приеме,inan,loct,masc,NOUN,1,344.0,sing,0.728661,0.485958,0.984812
918,вышло,2,2,6.0,3.0,344.0,NaN,NaN,neut,NaN,VERB,ROOT,вышло,NaN,NaN,neut,VERB,0,0.0,sing,0.511568,0.514919,0.978217
2212,в,3,3,5.0,8.0,344.0,NaN,NaN,NaN,NaN,PREP,case,использовании,inan,loct,neut,NOUN,0,0.0,NaN,0.477165,0.551943,0.871968
5897,и,4,4,6.0,7.0,344.0,NaN,NaN,NaN,NaN,CONJ,cc,выписывались,NaN,NaN,NaN,VERB,0,0.0,NaN,0.449110,0.523397,0.954007
6542,в,5,5,17.0,0.0,344.0,NaN,NaN,NaN,NaN,PREP,case,мозгу,inan,loc2,masc,NOUN,0,0.0,NaN,0.477165,0.462024,0.937972
8883,их,7,7,0.0,32.0,344.0,NaN,accs,NaN,3per,NPRO,obj,принимать,NaN,NaN,NaN,INFN,0,0.0,plur,0.477507,0.503339,0.988862
11513,отвратно,9,9,0.0,39.0,344.0,NaN,NaN,NaN,NaN,ADVB,advmod,перенесла,NaN,NaN,femn,VERB,0,0.0,NaN,0.547855,0.567049,0.990039
12936,будет,15,15,0.0,31.0,344.0,NaN,NaN,NaN,3per,VERB,advcl,проходить,NaN,NaN,NaN,INFN,0,0.0,sing,0.479168,0.616798,0.973237
15278,ковид,16,16,9.0,0.0,344.0,inan,nomn,masc,NaN,NOUN,conj,наименование,inan,nomn,neut,NOUN,0,0.0,sing,0.606161,0.483781,0.880903
16465,сказал,18,18,7.0,2.0,344.0,NaN,NaN,masc,NaN,VERB,acl:relcl,парень,anim,nomn,masc,NOUN,0,0.0,sing,0.484297,0.536376,0.985110


In [45]:

topic_num = 1
topic_df = df.loc[df['TOPIC_NUM'] == topic_num]
# номера предложений топика
sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num ].SENT_NUM))
sents_in_topic
# достать пост
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1)]
# достать конкретное предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)]
# достать токен из предложение конкретного поста
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)]


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
5,не,1,1,1.0,0.0,5.0,NaN,NaN,NaN,NaN,PRCL,advmod,знаю,NaN,NaN,NaN,VERB,0,0.0,NaN,0.457746,0.538269,0.947129
6,знаю,1,1,1.0,0.0,6.0,NaN,NaN,NaN,1per,VERB,ROOT,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.560695,0.538269,0.947129
7,стоило,1,1,1.0,0.0,7.0,NaN,NaN,neut,NaN,VERB,ccomp,знаю,NaN,NaN,NaN,VERB,0,0.0,sing,0.503683,0.538269,0.947129
8,ли,1,1,1.0,0.0,8.0,NaN,NaN,NaN,NaN,CONJ,advmod,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.509498,0.466043,0.947129
9,мне,1,1,1.0,0.0,9.0,NaN,datv,NaN,1per,NPRO,iobj,стоило,NaN,NaN,neut,VERB,0,0.0,sing,0.556107,0.466043,0.947129
10,писать,1,1,1.0,0.0,10.0,NaN,NaN,NaN,NaN,INFN,csubj,стоило,NaN,NaN,neut,VERB,0,0.0,NaN,0.505710,0.466043,0.947129


In [46]:
for i in df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 0)].values:
  print(i[-5])

0.0
0.0
0.0
0.0
0.0
0.0


In [47]:
# анафоры
df.loc[(df['TOPIC_NUM'] == 1) & (df['POST_NUM'] == 1) & (df['SENT_NUM'] == 1)
& (df['NER'] > 1)].values[0]

array(['лекарством', 1, 1, 1.0, 1.0, 21.0, 'inan', 'ablt', 'neut', nan,
       'NOUN', 'conj', 'нет', nan, nan, nan, 'PRED', 2, 0.0, 'sing',
       0.4964575171470642, 0.5259952545166016, 0.9471293091773987],
      dtype=object)

In [48]:
def rule_eval(my_X, my_y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(my_X)):
    if my_X[i] != 0:
      if my_X[i] == my_y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif my_X[i] == 0:
      if my_X[i] == my_y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')


In [49]:
df.loc[(df['POS'] == 5) &  (df['PERSON'] == 1)].iloc[30:40, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT


In [50]:
topics = list(set(df.TOPIC_NUM.values))
topics
# all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]

In [51]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize

mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
# mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [147]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_anaphors = sent.loc[(sent['POS'] == 5) &  (sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            # в противном случае присваиваем нулевой класс
            ann_clusters.append(anfr[-5])
            # и класс, который определило правило(антецедента из предложения)
            my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 540.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 2.0, 679.0, 0.0, 0.0, 0.0, 1483.0, 0.0, 1750.0, 2031.0, 2053.0, 0.0, 0.0, 0.0, 3125.0, 3270.0, 3619.0, 5.0, 114.0, 174.0, 365.0, 0.0, 527.0, 10.0, 27.0, 41.0, 41.0, 2.0, 135.0, 135.0, 218.0, 239.0, 256.0, 270.0, 270.0, 2.0, 2.0, 447.0, 521.0, 521.0, 521.0, 0.0, 0.0, 0.0, 2.0, 0.0, 1455.0, 1516.0, 1609.0, 1657.0, 1722.0, 1770.0, 0.0, 1905.0, 1984.0, 0.0, 0.0, 2071.0, 0.0, 164.0, 0.0, 410.0, 534.0, 0.0, 0.0, 707.0, 707.0, 801.0, 0.0, 871.0, 881.0, 1350.0, 0.0, 0.0, 0.0, 122.0, 165.0, 263.0, 513.0, 556.0, 654.0, 813.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1867.0, 2138.0, 2295.0, 305.0, 0.0, 0.0, 0.0, 0.0, 752.0]
---
[2.0, 536.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 658.0, 658.0, 829.0, 829.0, 829.0, 829.0, 183.0, 183.0, 183.0, 388.0, 446.0, 686.0, 778.0, 987.0, 987.0, 1490.0, 1582.0, 1750.0, 2031.0, 2053.0, 2344.0, 3.0, 3.0, 3125.0, 3270.0, 

In [148]:
rule_eval(my_clusters, ann_clusters)

52  true positive
26  false positive
14  true negative
28  false negative
---------
0.6666666666666666  presicion
0.65  recall
0.6582278481012659  f-score


In [149]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      # current_anaphors = sent.loc[(sent['POS'] == 5) &  (sent['NER'] > 1)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 4.0, 2.0, 0.0, 0.0, 344.0, 382.0, 2.0, 2.0, 440.0, 442.0, 2.0, 2.0, 540.0, 2.0, 3.0, 11.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 375.0, 434.0, 461.0, 649.0, 650.0, 658.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 1022.0, 1081.0, 1081.0, 1142.0, 1142.0, 0.0, 3653.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3653.0, 3653.0, 3653.0, 183.0, 0.0, 0.0, 0.0, 2.0, 2.0, 533.0, 551.0, 575.0, 593.0, 679.0, 686.0, 0.0, 3653.0, 0.0, 3653.0, 830.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3653.0, 3653.0, 3653.0, 987.0, 0.0, 1344.0, 1355.0, 1379.0, 1397.0, 2.0, 0.0, 1483.0, 1490.0, 0.0, 3653.0, 0.0, 0.0, 0.0, 1750.0, 1750.0, 2.0, 0.0, 1844.0, 0.0, 2.0, 3653.0, 0.0, 2031.0, 2031.0, 2053.0, 2053.0, 0.0, 3653.0, 2058.0, 2084.0, 2155.0, 0.0, 2.0, 3653.0, 0.0, 2156.0, 0.0, 2344.0, 2.0, 0.0, 2711.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 2766.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 3125.0, 3125.0, 0.0, 0.0, 3270.0, 3270.0, 3350.0, 3350.0, 3350.0, 0.0,

In [150]:
rule_eval(my_clusters, ann_clusters)

158  true positive
170  false positive
95  true negative
97  false negative
---------
0.4817073170731707  presicion
0.6196078431372549  recall
0.5420240137221269  f-score


In [172]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      # current_anaphors = sent.loc[(sent['NER'] > 1)]
      current_anaphors = sent.loc[(sent['POS'] == 5)& (df['PERSON']== 2)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[-1]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 540.0, 11.0, 11.0, 11.0, 108.0, 182.0, 182.0, 182.0, 182.0, 295.0, 349.0, 349.0, 558.0, 564.0, 658.0, 658.0, 658.0, 755.0, 829.0, 829.0, 829.0, 829.0, 942.0, 996.0, 996.0, 1205.0, 1211.0, 0.0, 0.0, 0.0, 0.0, 3653.0, 3653.0, 0.0, 0.0, 2.0, 679.0, 686.0, 0.0, 3653.0, 0.0, 0.0, 0.0, 0.0, 3653.0, 3653.0, 0.0, 0.0, 2.0, 1483.0, 1490.0, 0.0, 3653.0, 1750.0, 1844.0, 1844.0, 2031.0, 2031.0, 2053.0, 2053.0, 2156.0, 0.0, 0.0, 3653.0, 0.0, 0.0, 3125.0, 3270.0, 3270.0, 3488.0, 3488.0, 3511.0, 0.0, 3619.0, 5.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 527.0, 0.0, 0.0, 2.0, 0.0, 2.0, 0.0, 2.0, 0.0, 0.0, 527.0, 0.0, 0.0, 0.0, 0.0, 365.0, 0.0, 0.0, 527.0, 0.0, 10.0, 14.0, 27.0, 27.0, 41.0, 41.0, 2.0, 135.0, 135.0, 218.0, 226.0, 239.0, 243.0, 256.0, 256.0, 270.0, 270.0, 2.0, 2.0, 2.0, 447.0, 455.0, 2.0, 521.0, 521.0, 521.0, 521.0, 521.0, 521.0, 0.0, 0.0, 655.0, 0.0, 2.0, 2225.0, 2.0, 2225.0, 2.0, 2.0, 2.0, 2225.0, 1369.0, 1455.0, 1516.0, 1516.0, 1609.0, 1657.0, 1683.0, 1683.0, 1722.0, 1770.0, 1770

In [173]:
rule_eval(my_clusters, ann_clusters)

67  true positive
111  false positive
68  true negative
35  false negative
---------
0.37640449438202245  presicion
0.6568627450980392  recall
0.4785714285714285  f-score


In [169]:
 df.loc[(df['PERSON']== 2) & (df['POS']== 5)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,0,4,1,2,5,11,199,1,3,3,1,3,2.0,1,0.532177,0.360040,0.979649
566,он,1,1,9.0,3.0,566.0,0,1,1,2,5,12,231,0,0,1,0,3,536.0,1,0.512121,0.451582,0.979649
587,он,2,2,1.0,0.0,13.0,0,1,1,2,5,12,235,0,0,1,0,3,11.0,1,0.512121,0.550732,0.979649
594,он,2,2,1.0,1.0,20.0,0,1,1,2,5,12,242,0,0,0,0,3,11.0,1,0.512121,0.631055,0.979649
736,него,2,2,3.0,5.0,162.0,0,2,1,2,5,10,293,0,0,0,0,3,108.0,1,0.567449,0.582347,0.981903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16540,его,18,18,7.0,5.0,419.0,0,4,1,2,5,1,1588,0,0,0,5,0,0.0,1,0.532177,0.533587,0.985110
16770,он,20,20,0.0,11.0,181.0,0,1,1,2,5,12,1237,0,0,1,0,0,0.0,1,0.512121,0.506693,0.985110
17357,он,20,20,4.0,11.0,768.0,0,1,1,2,5,12,1237,0,0,1,0,2,0.0,1,0.512121,0.506693,0.985262
17731,он,20,20,4.0,39.0,1142.0,0,1,1,2,5,12,2270,0,0,1,0,0,0.0,1,0.512121,0.550634,0.985054


род

In [162]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      # current_anaphors = sent.loc[(sent['POS'] == 5)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                  anfr[8] == current_antecedents.GENDER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                      anfr[8] == current_antecedents.GENDER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 382.0, 2.0, 2.0, 440.0, 442.0, 2.0, 2.0, 540.0, 2.0, 3.0, 11.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 0.0, 0.0, 0.0, 649.0, 650.0, 658.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 356.0, 0.0, 0.0, 0.0, 533.0, 551.0, 575.0, 593.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 830.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1160.0, 0.0, 0.0, 1379.0, 1397.0, 2.0, 3653.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2155.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2711.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 2766.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3059.0, 3125.0, 3125.0, 0.0, 0.0, 3270.0, 3270.0, 3350.0, 3350.0, 3350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 5.0, 5.0, 2.0, 114.0, 114.0, 0.0, 174.0, 174.0, 0.0, 0.0, 0.0, 0.0, 0.

In [163]:
rule_eval(my_clusters, ann_clusters)

114  true positive
100  false positive
133  true negative
178  false negative
---------
0.5327102803738317  presicion
0.3904109589041096  recall
0.4505928853754941  f-score


In [ ]:
# def anaphore_in_sent(sent_num, topic_num) это потом
# изначально отталкиваемся от номеров топиков, итерация по ним
# topics = list(set(df.TOPIC_NUM.values)) это выше
anf_len = 0
ant_len = 0
ann_clusters = []
my_clusters = []
post_num = 0
for topic_num in topics:
  # print(topics, 'topics')
  # if post_num == 3:
  #   break
  # извлекаем посты топика
  posts_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].POST_NUM))
  # sents_in_topic = list(set(df.loc[df['TOPIC_NUM'] == topic_num].SENT_NUM))


  for post_num in posts_in_topic:
    sents_in_post = list(set(df.loc[(df['TOPIC_NUM'] == topic_num) \
                                    & (df['POST_NUM'] == post_num)].SENT_NUM))
    # if post_num == 3:
    #   break

    for sent_num in sents_in_post:

      sent = df.loc[(df['TOPIC_NUM'] == topic_num)  & (df['SENT_NUM'] == sent_num) \
                    & (df['POST_NUM'] == post_num)]
      # собираем анафоры и антецеденты
      current_anaphors = sent.loc[(sent['NER'] > 1)]
      # current_anaphors = sent.loc[(sent['POS'] == 5)]
      current_antecedents = sent.loc[(sent['NER'] == 1)]
      anf_len += len(current_anaphors)
      ant_len += len(current_antecedents)
      # если нет анафоров, продолжать не имеет смысла
      
      if not list(current_anaphors.TOKEN):
        continue

      # если в предложении есть один антецедент, самый простой случай
      if len(list(current_antecedents.TOKEN)) >= 1:
        for anfr in current_anaphors.values:
          # проверяем сходимость по числу
          if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                  anfr[8] == current_antecedents.GENDER.values[0]:
            # добавляем в список размеченный класс
            ann_clusters.append(anfr[-5])
            # и класс который определило правило(антецедента из предложения)
            my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
          else:
            topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
            current_antecedents = topic.loc[(topic['NER'] == 1) ]
            # проверяем, а антецеденты-то есть?
            if not current_antecedents.TOKEN.values.any():
                # присваиваем нулевой класс, если нет даже в топике
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                # для оценки
                my_clusters.append(0.0)
                continue
            # ну и если они есть
            # это можно убрать в отдельную функцию
            for anfr in current_anaphors.values:
              # проверяем сходимость по числу
              # try:
              if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                      anfr[8] == current_antecedents.GENDER.values[0]:
                # добавляем в список размеченный класс
                ann_clusters.append(anfr[-5])
                # и класс который определило правило(антецедента из предложения)
                my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
                # print(ann_clusters, my_clusters, 'am I here?')
              else:
                # в противном случае присваиваем нулевой класс
                ann_clusters.append(anfr[-5])
                # и класс, который определило правило(антецедента из предложения)
                my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)

      # если больше одного!!
          
      # если в предложении его нет =(
      elif len(list(current_antecedents.TOKEN)) == 0:
        # ищем в посте в предложениях выше
        post = df.loc[(df['TOPIC_NUM'] == topic_num) \
                      & (df['POST_NUM'] == post_num) \
                      & (df['SENT_NUM'] <= sent_num)]

        current_antecedents = post.loc[(post['NER'] == 1) ]

        # если его нету в посте, выбираем первый же антецедент из предыдущих постов
        # топика(сверху вниз)
        if not list(current_antecedents.TOKEN):
          topic = df.loc[(df['TOPIC_NUM'] == topic_num)]
          current_antecedents = topic.loc[(topic['NER'] == 1) ]
          # проверяем, а антецеденты-то есть?
          if not current_antecedents.TOKEN.values.any():
              # присваиваем нулевой класс, если нет даже в топике
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              # для оценки
              my_clusters.append(0.0)
              continue
          # ну и если они есть
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу
            # try:
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[0])
              # print(ann_clusters, my_clusters, 'am I here?')
            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)


        # если антецедент есть в посте раньше, выбираем первый с конца [::-1]
        else:
          # это можно убрать в отдельную функцию
          for anfr in current_anaphors.values:
            # проверяем сходимость по числу, если число вообще есть
          
            if anfr[-4] == current_antecedents.NUMBER.values[0] and\
                    anfr[8] == current_antecedents.GENDER.values[0]:
              # добавляем в список размеченный класс
              ann_clusters.append(anfr[-5])
              # и класс который определило правило(антецедента из предложения)
              my_clusters.append(current_antecedents.TOKEN_NUM.values[-1])

            else:
              # в противном случае присваиваем нулевой класс
              ann_clusters.append(anfr[-5])
              # и класс, который определило правило(антецедента из предложения)
              my_clusters.append(0.0)






      # print(list(current_anaphors.TOKEN), list(current_anaphors.TOKEN), list(current_anaphors.COREFERENCE_CLUSTER) )
      # print(list(current_antecedents.TOKEN), list(current_antecedents.TOKEN), list(current_antecedents.COREFERENCE_CLUSTER) )
print(my_clusters)
print('---')
print(ann_clusters)
print('---')
print(len(my_clusters), len(ann_clusters))
print(anf_len, ant_len)

[2.0, 4.0, 0.0, 0.0, 0.0, 0.0, 382.0, 2.0, 2.0, 440.0, 442.0, 2.0, 2.0, 540.0, 2.0, 3.0, 11.0, 11.0, 11.0, 46.0, 108.0, 108.0, 182.0, 182.0, 182.0, 182.0, 0.0, 0.0, 0.0, 649.0, 650.0, 658.0, 658.0, 658.0, 693.0, 755.0, 755.0, 829.0, 829.0, 829.0, 829.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 26.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 356.0, 0.0, 0.0, 0.0, 533.0, 551.0, 575.0, 593.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 830.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1160.0, 0.0, 0.0, 1379.0, 1397.0, 2.0, 3653.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1844.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2155.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2711.0, 2711.0, 2711.0, 2766.0, 2766.0, 2766.0, 2766.0, 0.0, 0.0, 0.0, 0.0, 0.0, 3059.0, 3125.0, 3125.0, 0.0, 0.0, 3270.0, 3270.0, 3350.0, 3350.0, 3350.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.0, 2.0, 5.0, 5.0, 2.0, 114.0, 114.0, 0.0, 174.0, 174.0, 0.0, 0.0, 0.0, 0.0, 0.

In [ ]:
rule_eval(my_clusters, ann_clusters)

114  true positive
100  false positive
133  true negative
178  false negative
---------
0.5327102803738317  presicion
0.3904109589041096  recall
0.4505928853754941  f-score


# **МОДЕЛИ.**

Факторизуем троковые данные, чтобы вместо строковых представлений классов были численные.

In [54]:
for_factorize = 'ANIMACY 	CASE	GENDER	PERSON	POS	DEPENDENCY	HEAD	HEAD_ANIMACY	HEAD_CASE	HEAD_GENDER	HEAD_POS NUMBER'.split()
for_factorize


['ANIMACY',
 'CASE',
 'GENDER',
 'PERSON',
 'POS',
 'DEPENDENCY',
 'HEAD',
 'HEAD_ANIMACY',
 'HEAD_CASE',
 'HEAD_GENDER',
 'HEAD_POS',
 'NUMBER']

In [55]:
_, lbl = pd.factorize(df.ANIMACY)
print(lbl, )
_

Int64Index([-1, 0, 1], dtype='int64')


array([0, 1, 2, ..., 0, 0, 0])

Делаем словарь с маппингом, чтобы можно было достать нужное значение по индексу. В значении -1 хранится None

In [56]:
mapping_dict = {}
for col_name in for_factorize:
  col, lbl = pd.factorize(df[col_name])
  df[col_name] = col
  mapping_dict[col_name] = lbl
mapping_dict  

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


{'ANIMACY': Int64Index([-1, 0, 1], dtype='int64'),
 'CASE': Int64Index([-1, 0, 1, 2, 3, 4, 5, 6, 7, 8], dtype='int64'),
 'DEPENDENCY': Int64Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
             17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
             34, 35],
            dtype='int64'),
 'GENDER': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD': Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             2972, 2973, 2974, 2975, 2976, 2977, 2978, 2979, 2980, 2981],
            dtype='int64', length=2982),
 'HEAD_ANIMACY': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD_CASE': Int64Index([-1, 0, 1, 2, 3, 4, 5, 6, 7], dtype='int64'),
 'HEAD_GENDER': Int64Index([-1, 0, 1, 2], dtype='int64'),
 'HEAD_POS': Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, -1, 14, 15, 16], dtype='int64'),
 'NUMBER': Int64Index([0, 1, -1, 2], dtype='int64'),
 'PERSON': Int64Index([0, -1, 1, 2], dtype=

In [57]:
df.loc[ (df['NER'] == 3.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
475,его,1,1,7.0,1.0,475.0,0,4,1,2,5,11,199,1,3,3,1,3,2.0,1,0.532177,0.360040,0.979649
566,он,1,1,9.0,3.0,566.0,0,1,1,2,5,12,231,0,0,1,0,3,536.0,1,0.512121,0.451582,0.979649
587,он,2,2,1.0,0.0,13.0,0,1,1,2,5,12,235,0,0,1,0,3,11.0,1,0.512121,0.550732,0.979649
594,он,2,2,1.0,1.0,20.0,0,1,1,2,5,12,242,0,0,0,0,3,11.0,1,0.512121,0.631055,0.979649
736,него,2,2,3.0,5.0,162.0,0,2,1,2,5,10,293,0,0,0,0,3,108.0,1,0.567449,0.582347,0.981903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14906,него,15,15,19.0,3.0,2314.0,0,2,1,2,5,21,2453,0,0,0,16,3,2295.0,1,0.567449,0.502688,0.928931
14967,он,16,16,1.0,4.0,33.0,0,1,1,2,5,12,2504,0,0,1,0,3,0.0,1,0.512121,0.651199,0.939065
16328,они,18,18,6.0,2.0,207.0,0,1,0,2,5,12,2507,0,0,0,9,3,0.0,0,0.463882,0.581616,0.927446
16391,его,18,18,6.0,5.0,270.0,0,4,1,2,5,1,1588,0,0,0,5,3,0.0,1,0.532177,0.533587,0.985110


In [58]:
df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 )]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,1,1,1,1,1,1,0,0,0,0,0,2,0.0,1,0.602498,0.528361,0.947129
2,рисперидон,1,1,0.0,0.0,2.0,2,1,1,1,1,2,1,1,1,1,1,1,2.0,1,0.735588,0.858878,0.947129
3,рисполепт,1,1,0.0,0.0,3.0,1,2,2,1,1,3,2,2,1,1,1,1,3.0,0,0.763361,0.562444,0.947129
4,сперидан,1,1,0.0,0.0,4.0,0,0,1,1,2,4,3,1,2,2,1,1,4.0,1,0.730097,0.628638,0.947129
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.496458,0.525995,0.947129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17918,вел,21,21,4.0,2.0,137.0,0,0,1,1,0,0,2961,0,0,1,0,1,136.0,1,0.512789,0.445758,0.985054
17929,соф,21,21,4.0,3.0,148.0,1,2,2,1,1,21,2964,1,2,1,1,1,148.0,0,0.533214,0.515160,0.985054
17930,даке,21,21,4.0,3.0,149.0,1,3,2,1,1,21,2945,1,2,2,1,1,149.0,1,0.530879,0.500351,0.985054
17937,соф,21,21,4.0,3.0,156.0,1,2,2,1,1,12,2961,0,0,1,0,1,156.0,0,0.533214,0.445758,0.985054


Разделим датасет на тестовый и тренировочный. for_train, for_test 

In [59]:
# train, test = train_test_split(df.loc[(df['NER'] == 1.0) | (df['NER'] == 2.0 ) | (df['NER'] == 3.0 )], test_size=0.2)
# train, test = train_test_split(df, test_size=0.2)
train, test = train_test_split(df.loc[(df['NER'] > 1 )], test_size=0.2)

In [60]:
train = train.reset_index()

In [61]:
test = test.reset_index()

In [62]:
test.head(5)

,index,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
0,7312,ад,5,5,35.0,2.0,1114.0,1,1,1,1,1,21,69,1,3,2,1,2,0.0,1,0.591204,0.494370,0.789907
1,7607,нейролептики,5,5,39.0,0.0,1409.0,1,1,1,1,1,17,737,1,1,1,1,2,0.0,0,0.683380,0.713682,0.878974
2,3343,таблетки,3,3,18.0,5.0,1475.0,1,4,2,1,1,4,274,1,1,1,1,2,1473.0,0,0.535919,0.457875,0.953173
3,3930,антипсихотиком,3,3,23.0,8.0,2062.0,2,6,1,1,1,19,798,0,0,0,0,2,2058.0,1,0.670544,0.491749,0.971506
4,15692,химиотерапии,16,16,17.0,3.0,758.0,1,2,2,1,1,21,142,2,1,1,1,2,0.0,1,0.622116,0.583157,0.947154


Делаем наборы для обучения моделей.

In [63]:
X, y, X_test, y_test = train.drop(['HEAD_VECT', 'NUMBER', 'TOKEN',  'GENDER', 'POS', 'IS_ANSWER', 'PERSON', \
                                   'TOPIC_NUM', 'CASE',   'DEPENDENCY',   'HEAD_ANIMACY',	'HEAD_CASE',\
                                   	'HEAD_GENDER',  'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18], \
train.COREFERENCE_CLUSTER, test.drop(['HEAD_VECT', 'NUMBER','TOKEN','GENDER', 'POS', 'IS_ANSWER', 'PERSON', 'TOPIC_NUM', \
                                      'CASE',  'DEPENDENCY',     'HEAD_ANIMACY',	'HEAD_CASE',\
                                      'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18],\
                                       test.COREFERENCE_CLUSTER

In [64]:
X.NER = [float(i) for i in X.NER]
X_test.NER = [float(i) for i in X_test.NER]
X.drop(['COREFERENCE_CLUSTER'], axis=1)

,index,POST_NUM,SENT_NUM,TOKEN_NUM,NER,TOKEN_VECT,SENT_VECT
0,5659,4.0,1.0,106.0,3.0,0.532177,0.954007
1,956,7.0,2.0,382.0,2.0,0.602498,0.978217
2,6010,7.0,3.0,457.0,2.0,0.476538,0.965493
3,2314,6.0,1.0,446.0,2.0,0.606009,0.834042
4,6445,11.0,0.0,247.0,3.0,0.532177,0.938670
...,...,...,...,...,...,...,...
285,8195,49.0,2.0,1997.0,3.0,0.460453,0.988862
286,12150,0.0,2.0,34.0,2.0,0.479098,0.985230
287,10591,15.0,1.0,2052.0,2.0,0.602207,0.991728
288,6180,9.0,0.0,627.0,2.0,0.443305,0.965493


In [65]:
y

0       96.0
1      375.0
2      449.0
3      446.0
4      243.0
       ...  
285      0.0
286     27.0
287      0.0
288      0.0
289    446.0
Name: COREFERENCE_CLUSTER, Length: 290, dtype: float64

Обучаем модели.

In [66]:
clf = tree.DecisionTreeClassifier(max_depth=2000, )
clf.fit(X, y)
clf.score(X_test, y_test)

0.5616438356164384

In [67]:
clf_entropy = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf_entropy.fit(X, y)
clf_entropy.score(X_test, y_test)

0.5753424657534246

In [68]:
randf_clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
randf_clf.fit(X, y)
randf_clf.score(X_test, y_test)

0.589041095890411

In [69]:

regr = linear_model.LinearRegression()
regr.fit(X, y)
regr.score(X_test, y_test)


1.0

In [70]:
svm_clf = svm.SVC(decision_function_shape='ovo', cache_size=500)
svm_clf.fit(X, y)
svm_clf.score(X_test, y_test)

0.3561643835616438

In [71]:

knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
knn_clf.score(X_test, y_test)

0.5205479452054794

In [72]:
# from sklearn.naive_bayes import MultinomialNB
# naive_clf = MultinomialNB().fit(X, y)


# **ОЦЕНКА МОДЕЛЕЙ**

In [73]:
def my_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y_test[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y_test[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')



In [74]:
pred = clf.predict(X_test)
pred_randf = randf_clf.predict(X_test)
pred_entropy = clf_entropy.predict(X_test)
pred_regr = regr.predict(X_test)
pred_svm = svm_clf.predict(X_test)
pred_knn = knn_clf.predict(X_test)
# naive_pred = naive_clf.predict(X_test)

In [75]:
# my_eval(list(naive_pred), list(y_test))

In [76]:
my_eval(list(pred_knn), list(y_test))

17  true positive
31  false positive
21  true negative
4  false negative
---------
0.3541666666666667  presicion
0.8095238095238095  recall
0.4927536231884058  f-score


In [77]:
my_eval(list(pred_entropy), list(y_test))

16  true positive
30  false positive
26  true negative
1  false negative
---------
0.34782608695652173  presicion
0.9411764705882353  recall
0.5079365079365079  f-score


In [78]:
my_eval(list(pred), list(y_test))

15  true positive
31  false positive
26  true negative
1  false negative
---------
0.32608695652173914  presicion
0.9375  recall
0.48387096774193544  f-score


In [79]:
my_eval(list(pred_randf), list(y_test))

17  true positive
29  false positive
26  true negative
1  false negative
---------
0.3695652173913043  presicion
0.9444444444444444  recall
0.53125  f-score


In [80]:
my_eval(list(pred_regr), list(y_test))

0  true positive
73  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [81]:
my_eval(list(pred_svm), list(y_test))

0  true positive
0  false positive
26  true negative
47  false negative
---------
0  presicion
0.0  recall
0  f-score


In [82]:
# df['TOPIC_NUM'] == 1
# df.loc[df['TOPIC_NUM'] == 1]

In [83]:
df.drop(['TOKEN','GENDER','NUMBER', 'POS', 'HEAD_VECT', 'POST_NUM', 'IS_ANSWER', 'PERSON', \
         'TOPIC_NUM', 'CASE', 'SENT_NUM', 'DEPENDENCY',  'HEAD_ANIMACY',	'HEAD_CASE',	\
         'HEAD_GENDER', 'ANIMACY',	'HEAD',	'HEAD_POS'], axis=1).iloc[:,:18]

,TOKEN_NUM,NER,COREFERENCE_CLUSTER,TOKEN_VECT,SENT_VECT
0,0.0,0,0.0,0.497393,0.947129
1,1.0,2,0.0,0.602498,0.947129
2,2.0,1,2.0,0.735588,0.947129
3,3.0,1,3.0,0.763361,0.947129
4,4.0,1,4.0,0.730097,0.947129
...,...,...,...,...,...
17995,214.0,0,0.0,0.434102,0.985054
17996,215.0,0,0.0,0.626010,0.985054
17997,216.0,0,0.0,0.477165,0.985054
17998,217.0,0,0.0,0.520455,0.985054


In [84]:
len(df.loc[(df['POS'] == 5) & (df['NER'] > 1)].TOKEN)

120

In [85]:
df.loc[(df['COREFERENCE_CLUSTER'] != 0) & (df['NER'] > 1)]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
366,препарата,1,1,2.0,5.0,366.0,1,2,1,1,1,21,154,1,2,2,1,2,344.0,1,0.532454,0.435295,0.984812
393,препарата,1,1,3.0,1.0,393.0,1,2,1,1,1,21,164,1,4,3,1,2,382.0,1,0.532454,0.424344,0.984812
421,препарат,1,1,5.0,0.0,421.0,1,1,1,1,1,0,1,1,1,1,1,2,2.0,1,0.602498,0.858878,0.984812
429,препарат,1,1,6.0,0.0,429.0,1,1,1,1,1,12,179,0,0,0,0,2,2.0,1,0.602498,0.499828,0.984812
446,последнему,1,1,6.0,2.0,446.0,0,3,1,1,10,7,184,0,0,0,0,2,442.0,1,0.607927,0.530991,0.984812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14440,его,15,15,14.0,2.0,1848.0,0,4,1,2,5,1,1175,0,0,0,0,3,1827.0,1,0.532177,0.566555,0.950498
14478,него,15,15,14.0,6.0,1886.0,0,2,1,2,5,21,2453,0,0,0,16,3,1867.0,1,0.567449,0.502688,0.938599
14733,него,15,15,17.0,8.0,2141.0,0,2,1,2,5,10,2278,0,0,2,0,3,2138.0,1,0.567449,0.558731,0.945328
14906,него,15,15,19.0,3.0,2314.0,0,2,1,2,5,21,2453,0,0,0,16,3,2295.0,1,0.567449,0.502688,0.928931


In [86]:
int(df.loc[(df['TOPIC_NUM'] == 1) & (df['POS'] == 5)].values[0][4])
df.loc[(df['TOPIC_NUM'] == 1) & (df['POS'] == 5)].values[0][5]

9.0

# **ПОПАРНОЕ ИЗВЛЕЧЕНИЕ**

In [87]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [88]:
df.loc[(df['TOPIC_NUM'] == 1) & \
              (df['NER'] == 1) & \
              (df['SENT_NUM'] <= 4.0) & \
              (df['POST_NUM'].isin([0.0, 2.0, 3.0]))]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
2,рисперидон,1,1,0.0,0.0,2.0,2,1,1,1,1,2,1,1,1,1,1,1,2.0,1,0.735588,0.858878,0.947129
3,рисполепт,1,1,0.0,0.0,3.0,1,2,2,1,1,3,2,2,1,1,1,1,3.0,0,0.763361,0.562444,0.947129
4,сперидан,1,1,0.0,0.0,4.0,0,0,1,1,2,4,3,1,2,2,1,1,4.0,1,0.730097,0.628638,0.947129
344,рисполепта,1,1,2.0,4.0,344.0,1,1,2,1,1,21,147,1,6,1,1,1,344.0,1,0.728661,0.485958,0.984812
382,рисперидон,1,1,3.0,0.0,382.0,2,1,1,1,1,1,67,0,0,0,0,1,382.0,1,0.735588,0.583913,0.984812
394,флюанксол,1,1,3.0,2.0,394.0,1,1,1,1,1,12,166,0,0,1,0,1,394.0,1,0.710806,0.474421,0.984812


In [89]:
topics = list(set(df.TOPIC_NUM.values))
topics


pair_anaphors = []
pair_antecedents = []
relation_list = []
counter = 0
ants_with_anaph = 0
len_anapfors = 0
for topic_num in topics:

  all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] > 1)]
  # all_anaphors = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['POS'] == 5)]

  all_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & (df['NER'] == 1)]
  
  len_anapfors += len(all_anaphors)
  for anaphor in all_anaphors.values:
    anapor_sent = anaphor[4]
    anapor_post = anaphor[3]
    local_possible_antecedents = df.loc[(df['TOPIC_NUM'] == topic_num) & \
                                        (df['NER'] == 1) & \
                                        (df['SENT_NUM'] <= anapor_sent) & \
                                        (df['POST_NUM'].isin([0.0, anapor_post]))]
    # чтобы примеры влезли в память, и модель не переобучилась на антипримерах,
    # возьмем один положительный и два отрицательных класса для каждого анафора
    counter = 0
    is_relation = 0
    for antecedent in local_possible_antecedents.values:
      if not antecedent.any():
        continue

      # print(anaphor[-5], antecedent[5])
      if anaphor[-5] == antecedent[5]:
        ants_with_anaph += 1
        is_relation = 1
        # такие срезы, чтобы убрать лишние данные, и в том числе разметку с ответами
        pair_anaphors.append(list(anaphor)[3:6]  + list(anaphor)[17:18]\
                              + list(anaphor)[19:22])
        
        pair_antecedents.append(list(antecedent)[3:6]  + list(antecedent)[17:18]\
                              + list(antecedent)[19:22])
        
        relation_list.append(is_relation)
      else:
        # if counter != 1:
        is_relation = 0

        pair_anaphors.append(list(anaphor)[3:6]  + list(anaphor)[17:18]\
                              + list(anaphor)[19:22])
        
        pair_antecedents.append(list(antecedent)[3:6]  + list(antecedent)[17:18]\
                              + list(antecedent)[19:22])
        
        relation_list.append(is_relation)
        counter = 1

  # counter += 1
print(len_anapfors)
print('---')
print(ants_with_anaph, 'ss')
print(pair_anaphors[0][0], pair_antecedents[0][0], relation_list[0])
len(pair_anaphors), len(pair_antecedents), len(relation_list)

363
---
188 ss
0.0 0.0 0


(1874, 1874, 1874)

In [90]:
# можно посмотреть, что получилось
relation_list[:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]

In [91]:
pair_anaphors[0]

[0.0, 0.0, 1.0, 2, 1, 0.602497935295105, 0.5283609628677368]

In [92]:
# проверим, третьим элементом должен быть номер токена
len([i for i in relation_list if i==1])

188

In [93]:
len([i for i in relation_list if i==0])

1686

In [94]:
# все сходится. мы взяли колонки с номерами поста, предложения и токена,\
# колонку NER, число, и вектора
df.loc[df.TOKEN_NUM == 21.0].iloc[[0]]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.496458,0.525995,0.947129


In [95]:
cosine = [cosine_similarity([pair_anaphors[i]], [pair_antecedents[i]])[0][0] for i in range(len(pair_anaphors))]
# pair_df = pd.DataFrame(columns=['anaphs', 'ants', 'rel'])
pair_df = pd.DataFrame(columns=['cosine', 'rel'])

# pair_df['anaphs'] = pair_anaphors
# pair_df['ants'] = pair_antecedents
pair_df['cosine'] = cosine
pair_df['rel'] = relation_list

print(pair_df.loc[pair_df['cosine'] > 1])
print(pair_df.loc[pair_df['rel'] > 1])
pair_df 

Empty DataFrame
Columns: [cosine, rel]
Index: []
Empty DataFrame
Columns: [cosine, rel]
Index: []


,cosine,rel
0,0.848099,0
1,0.676634,0
2,0.693146,0
3,0.801872,0
4,0.939316,0
...,...,...
1869,0.999987,0
1870,0.999987,0
1871,0.999996,0
1872,0.999996,0


In [96]:
pair_df.loc[pair_df.rel != 0]

,cosine,rel
16,0.999994,1
20,0.999993,1
21,0.744928,1
24,0.744574,1
31,0.999998,1
...,...,...
1777,0.999998,1
1810,0.999998,1
1825,0.999999,1
1827,0.999999,1


In [97]:
# train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
# train = train.reset_index()
# test = test.reset_index()

train, test = train_test_split(pair_df, test_size=0.3, shuffle=True)
train = train.reset_index()
test = test.reset_index()


In [98]:
train

,index,cosine,rel
0,470,1.000000,0
1,337,0.999998,0
2,1720,0.978166,0
3,847,0.768457,0
4,13,0.745030,0
...,...,...,...
1306,1259,0.999678,0
1307,591,0.999998,0
1308,630,0.999999,0
1309,239,0.999996,0


In [99]:
train.iloc[:1, 1:2]

,cosine
0,1.0


In [100]:
test.iloc[:1, 1:2]

,cosine
0,0.801473


In [101]:
X, y, X_test, y_test = train.iloc[:, 1:2], \
                    train.rel, test.iloc[:, 1:2], test.rel

In [102]:
def pair_eval(X_predicted, y):
  
  tp = []
  fp = []
  tn = []
  fn = []

  for i in range(len(X_predicted)):
    if X_predicted[i] != 0.0:
      if X_predicted[i] == y[i]:
        tp.append(i)
      else:
        fp.append(i)
    # берем только те синглтоны, которые относятся к номинативным или прономинальным группам.
    elif X_predicted[i] == 0.0:
      if X_predicted[i] == y[i]:
        tn.append(i)
      else:
        fn.append(i)
  try:
    P = len(tp)/(len(tp) + len(fp))
  except ZeroDivisionError:
    P = 0
  try:
    R = len(tp)/(len(tp) + len(fn))
  except ZeroDivisionError:
    R = 0
  try:
    F = 2*P*R/(P+R)
  except ZeroDivisionError:
    F = 0

  print(len(tp), ' true positive')
  print(len(fp), ' false positive')
  print(len(tn), ' true negative')
  print(len(fn), ' false negative')
  print('---------')
  print(P,' presicion')
  print(R, ' recall')
  print(F, ' f-score')

In [103]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
563  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [104]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
6  true positive
21  false positive
484  true negative
52  false negative
---------
0.2222222222222222  presicion
0.10344827586206896  recall
0.1411764705882353  f-score


In [105]:
dtclf = tree.DecisionTreeClassifier(max_depth=2000, )
dtclf.fit(X, y)


print('-----')
pred = dtclf.predict(X_test)
pair_eval(pred, y_test)


-----
2  true positive
17  false positive
488  true negative
56  false negative
---------
0.10526315789473684  presicion
0.034482758620689655  recall
0.051948051948051945  f-score


In [106]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
2  true positive
17  false positive
488  true negative
56  false negative
---------
0.10526315789473684  presicion
0.034482758620689655  recall
0.051948051948051945  f-score


In [107]:
clf = RandomForestClassifier(max_depth=3000, n_estimators=600, bootstrap=True, )
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
rule_eval(pred, y_test)

-----
2  true positive
19  false positive
486  true negative
56  false negative
---------
0.09523809523809523  presicion
0.034482758620689655  recall
0.050632911392405056  f-score


In [108]:
# knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1).fit(X, y)
# knn_clf.score(X_test, y_test)

In [109]:
# clf = tree.DecisionTreeClassifier(max_depth=2000, )
# clf.fit(X, y)
# clf.score(X_test, y_test)

# **Извлечение номинативных анафоров**

In [110]:
df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :]

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
16,проблем,1,1,1.0,1.0,16.0,1,2,2,1,1,12,7,0,0,0,3,0,0.0,0,0.522109,0.525995,0.947129
19,частности,1,1,1.0,1.0,19.0,1,5,2,1,1,15,10,0,0,0,4,0,0.0,1,0.557549,0.439704,0.947129
24,мере,1,1,1.0,2.0,24.0,1,3,2,1,1,15,12,0,0,0,4,0,0.0,1,0.588494,0.532547,0.947129
25,врач,1,1,1.0,2.0,25.0,2,1,1,1,1,12,11,0,0,0,0,0,0.0,1,0.596931,0.478720,0.947129
35,детства,1,1,1.0,3.0,35.0,1,2,3,1,1,10,15,0,0,0,0,0,0.0,1,0.484531,0.328795,0.947129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
849,месяц,2,2,5.0,1.0,275.0,1,4,1,1,1,10,324,0,0,0,5,0,0.0,1,0.569863,0.520403,0.978217
853,манию,2,2,5.0,1.0,279.0,1,4,2,1,1,10,326,0,0,2,0,0,0.0,1,0.525508,0.565204,0.978217
856,мг,2,2,5.0,1.0,282.0,1,1,1,1,1,10,326,0,0,2,0,0,0.0,1,0.576997,0.565204,0.978217
859,началом,2,2,5.0,2.0,285.0,1,6,3,1,1,10,328,0,0,2,0,0,0.0,1,0.544176,0.462819,0.978217


In [111]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)] 

,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
1,препарат,1,1,0.0,0.0,1.0,1,1,1,1,1,1,0,0,0,0,0,2,0.0,1,0.602498,0.528361,0.947129
21,лекарством,1,1,1.0,1.0,21.0,1,6,3,1,1,4,7,0,0,0,3,2,0.0,1,0.496458,0.525995,0.947129
217,таблетки,1,1,1.0,21.0,217.0,1,4,2,1,1,10,101,0,0,2,0,2,0.0,0,0.535919,0.682118,0.947129
295,таблетки,1,1,2.0,0.0,295.0,1,4,2,1,1,10,101,0,0,2,0,2,0.0,0,0.535919,0.682118,0.984812
366,препарата,1,1,2.0,5.0,366.0,1,2,1,1,1,21,154,1,2,2,1,2,344.0,1,0.532454,0.435295,0.984812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15651,вакцина,16,16,16.0,0.0,717.0,1,1,2,1,1,21,2642,1,1,3,1,2,0.0,1,0.564049,0.474780,0.942362
15692,химиотерапии,16,16,17.0,3.0,758.0,1,2,2,1,1,21,142,2,1,1,1,2,0.0,1,0.622116,0.583157,0.947154
15751,химиотерапий,16,16,18.0,1.0,817.0,1,2,2,1,1,21,2609,1,2,2,1,2,0.0,0,0.688022,0.653716,0.937090
15873,вакцина,16,16,19.0,0.0,939.0,1,1,2,1,1,21,2642,1,1,3,1,2,0.0,1,0.564049,0.474780,0.865680


In [112]:
df.loc[(df['POS'] == 1) & (df['NER'] > 1)].TOKEN.values

array(['препарат', 'лекарством', 'таблетки', 'таблетки', 'препарата',
       'препарата', 'препарат', 'препарат', 'препарат', 'препарат',
       'препарат', 'препарата', 'препарат', 'препарат', 'препарата',
       'препарат', 'препарат', 'препарата', 'препарата', 'препарат',
       'препарат', 'препарата', 'сибазона', 'препарат', 'препарат',
       'препарата', 'лекарства', 'препаратов', 'правозащитники',
       'препарата', 'препарата', 'препарат', 'препарат', 'препарата',
       'лекарства', 'препаратов', 'препарата', 'препарата', 'таблетки',
       'препарата', 'препарат', 'препараты', 'препарата', 'препараты',
       'антипсихотиком', 'препаратов', 'препараты', 'антидепресантом',
       'препаратов', 'препарат', 'препараты', 'лекарств', 'препарата',
       'препарат', 'препарата', 'препарат', 'нейролептика', 'нормотимики',
       'препараты', 'дженериков', 'препарата', 'препарат', 'препарат',
       'средство', 'нейролептиков', 'препарату', 'препарату',
       'препаратов', 'препар

In [113]:
# df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:209, :].append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)])
df.loc[(df['POS'] == 1) & (df['NER'] > 1)]
np.mean(df.loc[(df['POS'] == 1) & (df['NER'] > 1)].CASE)

2.0669642857142856

In [114]:
print(len(df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]))
df.loc[(df['POS'] != 1) & (df['POS'] != 5) & (df['NER'] > 1) ]

19


,TOKEN,IS_ANSWER,TOPIC_NUM,POST_NUM,SENT_NUM,TOKEN_NUM,ANIMACY,CASE,GENDER,PERSON,POS,DEPENDENCY,HEAD,HEAD_ANIMACY,HEAD_CASE,HEAD_GENDER,HEAD_POS,NER,COREFERENCE_CLUSTER,NUMBER,TOKEN_VECT,HEAD_VECT,SENT_VECT
446,последнему,1,1,6.0,2.0,446.0,0,3,1,1,10,7,184,0,0,0,0,2,442.0,1,0.607927,0.530991,0.984812
3539,препарат.точнее,3,3,21.0,0.0,1671.0,0,0,0,1,13,0,691,0,0,0,15,2,3.0,2,0.656030,0.865849,0.961899
5222,новейший,3,3,38.0,5.0,3354.0,0,1,1,1,10,0,1085,0,1,1,8,2,3350.0,1,0.614533,0.602314,0.954007
6857,тиапридал,5,5,23.0,6.0,659.0,0,0,1,1,0,12,958,0,0,0,0,2,659.0,1,0.724291,0.512564,0.926724
8834,это,7,7,0.0,25.0,295.0,0,0,0,1,3,12,941,0,0,0,9,3,293.0,2,0.464107,0.467143,0.988862
8928,хт,7,7,1.0,2.0,389.0,0,0,0,1,16,21,1634,1,1,1,1,2,0.0,2,0.556543,0.513509,0.988862
8987,хт,7,7,1.0,8.0,448.0,0,0,0,1,16,21,1651,1,2,3,1,2,0.0,2,0.556543,0.579346,0.988862
9086,хт,7,7,1.0,13.0,547.0,0,0,0,1,16,10,229,0,0,0,5,2,0.0,2,0.556543,0.578981,0.988862
9497,ксф,7,7,1.0,57.0,958.0,0,0,0,1,16,12,1791,0,0,0,0,2,0.0,2,0.497770,0.645442,0.988862
9935,хт,7,7,7.0,0.0,1396.0,0,0,0,1,16,21,1634,1,1,1,1,2,0.0,2,0.556543,0.513509,0.991728


In [115]:
med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:2409, :]\
            .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
            loc[:,['TOKEN_VECT', 'CASE', 'HEAD_VECT', 'NER']]


# med_df = df.loc[(df['POS'] == 1) &  (df['NER'] == 0)].iloc[:1409, :]\
#             .append(df.loc[(df['POS'] == 1) & (df['NER'] > 1)]).\
#             loc[:,['TOKEN_VECT', 'HEAD_VECT', 'SENT_VECT', 'NER']]

In [116]:
med_df

,TOKEN_VECT,CASE,HEAD_VECT,NER
16,0.522109,2,0.525995,0
19,0.557549,5,0.439704,0
24,0.588494,3,0.532547,0
25,0.596931,1,0.478720,0
35,0.484531,2,0.328795,0
...,...,...,...,...
15651,0.564049,1,0.474780,2
15692,0.622116,2,0.583157,2
15751,0.688022,2,0.653716,2
15873,0.564049,1,0.474780,2


In [117]:
train, test = train_test_split(med_df, test_size=0.3, shuffle=True)
train = train.reset_index().drop(['index'], axis=1)
test = test.reset_index().drop(['index'], axis=1)
X, y, X_test, y_test = train.drop(['NER'], axis=1), \
                    train.NER, test.drop(['NER'], axis=1), test.NER

In [118]:
# pred = knn_clf.predict(X_test)
# pair_eval(pred, y_test)

In [119]:
X

,TOKEN_VECT,CASE,HEAD_VECT
0,0.470600,2,0.526911
1,0.583649,1,0.577024
2,0.602498,1,0.620040
3,0.532135,4,0.504939
4,0.499167,1,0.450363
...,...,...,...
1838,0.543558,2,0.666048
1839,0.598140,5,0.574442
1840,0.508841,2,0.400994
1841,0.503691,1,0.556451


In [120]:
y

0       0
1       0
2       2
3       0
4       0
       ..
1838    0
1839    0
1840    0
1841    0
1842    0
Name: NER, Length: 1843, dtype: int64

In [121]:
len([i for i in y if i == 2])

147

In [122]:
y_test

0      2
1      0
2      0
3      0
4      0
      ..
785    0
786    0
787    0
788    0
789    0
Name: NER, Length: 790, dtype: int64

In [123]:
len([i for i in y_test if i == 2])

75

In [124]:
regr = linear_model.LinearRegression()
regr.fit(X, y)


print('-----')
pred = regr.predict(X_test)
pair_eval(pred, y_test)

-----
0  true positive
790  false positive
0  true negative
0  false negative
---------
0.0  presicion
0  recall
0  f-score


In [125]:
knn_clf = KNeighborsClassifier(n_neighbors=10, weights='distance',algorithm='ball_tree',leaf_size=5, metric='minkowski', p=1)
knn_clf.fit(X, y)


print('-----')
pred = knn_clf.predict(X_test)
pair_eval(pred, y_test)

-----
43  true positive
9  false positive
705  true negative
33  false negative
---------
0.8269230769230769  presicion
0.5657894736842105  recall
0.671875  f-score


In [126]:
clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=2000)
clf.fit(X, y)


print('-----')
pred = clf.predict(X_test)
pair_eval(pred, y_test)

-----
53  true positive
23  false positive
691  true negative
23  false negative
---------
0.6973684210526315  presicion
0.6973684210526315  recall
0.6973684210526315  f-score


In [127]:
topics_sum = 0
for i in topics:
  if df.loc[df['TOPIC_NUM']==i].POST_NUM.any():
    topics_sum += df.loc[df['TOPIC_NUM']==i].POST_NUM.values[-1]
    # print(df.loc[df['TOPIC_NUM']==i].POST_NUM.values[-1])
topics_sum

240.0